In [1]:
import numpy as np
import re
import csv
import os
from PIL import Image
import PIL
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
#from tensorflow.keras.preprocessing import image
#from tensorflow.keras.layers import GlobalAveragePooling2D
import tensorflow as tf
import keras


In [2]:
#for i in range (len(imageset)):
    #imageset[i] = os.path.join(directory, imageset[i])
#PIL.Image.open(imageset[i])

In [3]:
#images in the datatset are separate into these folders.
folders = ["3DPeS/archive", "CAVIAR4REID/archive", "CUHK/archive", "GRID/archive", "i-LID/archive", "MIT/archive", "PRID/archive", "SARC3D/archive", "TownCentre/archive", "VIPeR/archive"]
#Creating lists for the image set and label set.
fullimageset = []
fulllabelset = []

#Iterating over the folders to add images and labels to their lists.
for directory in folders:
    #Finding and reading the label file in the folder.
    #Sorting numerically.
    labelfile = os.path.join(directory, "Label.txt")
    file = open (labelfile)
    lines = file.readlines()
    lines = sorted(lines, key=lambda s: int(re.findall(r'\d+', s)[0]))

    labels = np.ndarray(shape = len(lines))
    labelnames = np.empty(shape = len(lines))
    
    #Substrings within the label files used for each age group.
    sub1 = "personalLess15"
    sub2 = "personalLess30"
    sub3 = "personalLess45"
    sub4 = "personalLess60"
    sub5 = "personalLarger60"

    #Store the corresponding age group in labels.
    # -1 if age group is not specified.
    for i in range (len(lines)):
        #if sub1 in lines[i]:
            #labels[i] = 0
        if sub2 in lines[i]:
            labels[i] = 0
        elif sub3 in lines[i]:
            labels[i] = 1
        elif sub4 in lines[i]:
            labels[i] = 2
        elif sub5 in lines[i]:
            labels[i] = 3
        else:
            labels[i] = -1

    #Obtaining the image name of each label, present at the start of each line in the label file.
    #Removing ".png" from the end of the name for CUHK folder.
    with open(labelfile) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=' ')
        line_count = 0
        for row in csv_reader:
            if directory == "CUHK/archive":
                row[0] = row[0][:4]
            labelnames[line_count] = row[0]
            line_count += 1
    labelnames.sort()

    
    # Creating a dictionary where the keys are the image names and values are label values.
    dictionary = {}

    for i in range (len(labels)):
        dictionary[int(labelnames[i])] = int(labels[i])


    #Obtaining all images in the folder.
    #Removing the last file in the list, Labels.txt.
    #Second list used to store only images with labels value != -1
    imageset = os.listdir(directory)
    imageset.pop(-1)
    newimageset = []
    labelset = []
    
    #Each directory has a different numbering format
    #Here the name of each image is being obtained
    #If an age label exists, the image and label are added to the set
    if directory == "TownCentre/archive" or directory == "3DPeS/archive":
        for i in range (len(imageset)):
            index = imageset[i][:3]
            if index[2] == "_":
                index = index[:2]
            elif index[1] == "_":
                index = index[:1]
            if dictionary.get(int(index)) != -1:
                newimageset.append(os.path.join(directory,imageset[i]))
                labelset.append(dictionary.get(int(index)))
    elif directory == "SARC3D/archive" or directory == "VIPeR/archive":
        for i in range (len(imageset)):
            index = imageset[i][:3]
            if dictionary.get(int(index)) != -1:
                newimageset.append(os.path.join(directory,imageset[i]))
                labelset.append(dictionary.get(int(index)))
                #labelset[i] = dictionary.get(int(index))
    elif directory == "MIT/archive":
        for i in range (len(imageset)):
            index = imageset[i][:5]
            if dictionary.get(int(index)) != -1:
                newimageset.append(os.path.join(directory,imageset[i]))
                labelset.append(dictionary.get(int(index)))
    else:

        for i in range (len(imageset)):
            index = imageset[i][:4]
            if dictionary.get(int(index)) != -1:
                newimageset.append(os.path.join(directory,imageset[i]))
                labelset.append(dictionary.get(int(index)))
    
    #At the end of the loop, elements are appended to the full image and label sets
    for i in range (len(newimageset)):
        imageset[i] = os.path.join(directory, imageset[i])
        fullimageset.append(newimageset[i])
        fulllabelset.append(labelset[i])





In [4]:
#Showing the number of images for each age range
fifteen = 0
thirty = 0
fortyfive = 0
sixty = 0
sixtyplus = 0

for i in fulllabelset:
    if i == 0: fifteen +=1
    if i == 1: thirty +=1
    if i == 2: fortyfive +=1
    if i == 3: sixty +=1
    if i == 4: sixtyplus +=1

print(fifteen)
print(thirty)
print(fortyfive)
print(sixty)
print(sixtyplus) 

9443
6246
1943
1170
0


In [5]:
#Splitting the sets into training and test sets
X_train, X_test, y_train, y_test = train_test_split(fullimageset, fulllabelset, test_size=0.33, random_state=42)

#Converting the labels to strings to convert to pandas dataframe
y_trains = []
y_tests = []


for i in range (len(y_train)):
        #if int(y_train[i]) == 0:
            #y_trains.append(sub1)
        if int(y_train[i]) == 0:
            y_trains.append(sub2)
        elif int(y_train[i]) == 1:
            y_trains.append(sub3)
        elif int(y_train[i]) == 2:
            y_trains.append(sub4)
        elif int(y_train[i]) == 3:
            y_trains.append(sub5)
        else:
            print(y_train[i])

            
for i in range (len(y_test)):
        #if int(y_test[i]) == 0:
            #y_tests.append(sub1)
        if int(y_test[i]) == 0:
            y_tests.append(sub2)
        elif int(y_test[i]) == 1:
            y_tests.append(sub3)
        elif int(y_test[i]) == 2:
            y_tests.append(sub4)
        elif int(y_test[i]) == 3:
            y_tests.append(sub5)


            


In [6]:
#Convert datasets to pandas dataframe
traindf = pd.DataFrame({"Images":X_train, "Labels":y_trains})
testdf = pd.DataFrame({"Images":X_test, "Labels":y_tests})

In [7]:
print(traindf)
print(testdf)

                                     Images            Labels
0                     CUHK/archive\1683.png    personalLess30
1           TownCentre/archive\154_3174.jpg    personalLess45
2                   PRID/archive\1255_b.png  personalLarger60
3           TownCentre/archive\154_3264.jpg    personalLess45
4                   PRID/archive\0541_b.png    personalLess45
...                                     ...               ...
12592       TownCentre/archive\120_2054.jpg    personalLess60
12593       TownCentre/archive\139_3064.jpg  personalLarger60
12594                 CUHK/archive\3165.png    personalLess45
12595  3DPeS/archive\6_513_FRAME_41_RGB.bmp    personalLess30
12596          TownCentre/archive\4_210.jpg    personalLess45

[12597 rows x 2 columns]
                               Images            Labels
0      TownCentre/archive\95_1716.jpg    personalLess60
1               CUHK/archive\0134.png    personalLess30
2      TownCentre/archive\77_1382.jpg    personalLess60
3     

In [8]:
#delete
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    validation_split=0.2)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

In [9]:
#Converting labels from strings to one hot encoding
labeldic = { "personalLess30": 0, "personalLess45":1, "personalLess60":2, "personalLarger60":3}

intytrains = []
for i in range (len(y_trains)):
    intytrains.append(labeldic.get(y_trains[i]))
labels_train = tf.keras.utils.to_categorical(intytrains)

intytest = []
for i in range (len(y_tests)):
    intytest.append(labeldic.get(y_tests[i]))
labels_test = tf.keras.utils.to_categorical(intytest)

In [10]:
#Defining training and testing generators
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(dataframe=traindf, directory=None, x_col="Images", y_col="Labels", target_size=(128, 48), batch_size= 8, class_mode='categorical')

test_generator = test_datagen.flow_from_dataframe(dataframe=testdf, directory=None, x_col="Images", y_col="Labels", target_size=(128, 48), batch_size= 8,
                                                           class_mode='categorical')


Found 12597 validated image filenames belonging to 4 classes.
Found 6205 validated image filenames belonging to 4 classes.


In [11]:
from keras import backend as K
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

In [12]:
#base model
ptmodel = tf.keras.applications.resnet50.ResNet50(
                    input_shape=(128,48, 3),
                    include_top=False,
                    weights='imagenet',
                    pooling='avg')

In [13]:
#Defining model and layers
inputs = ptmodel.input
input_shape = (128, 48, 3)
x = tf.keras.layers.Dense(128, activation='relu')(ptmodel.output)
x = tf.keras.layers.Dense(50, activation='relu')(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 48, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 54, 3)   0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 24, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 24, 64)   256         ['conv1_conv[0][0]']         

 conv2_block3_2_conv (Conv2D)   (None, 32, 12, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 32, 12, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 32, 12, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 32, 12, 256)  16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_blo

 conv3_block3_2_conv (Conv2D)   (None, 16, 6, 128)   147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 16, 6, 128)  512         ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 16, 6, 128)  0           ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 16, 6, 512)   66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_blo

 conv4_block2_2_conv (Conv2D)   (None, 8, 3, 256)    590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 8, 3, 256)   1024        ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 8, 3, 256)   0           ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_3_conv (Conv2D)   (None, 8, 3, 1024)   263168      ['conv4_block2_2_relu[0][0]']    
                                                                                                  
 conv4_blo

 conv4_block5_2_relu (Activatio  (None, 8, 3, 256)   0           ['conv4_block5_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_3_conv (Conv2D)   (None, 8, 3, 1024)   263168      ['conv4_block5_2_relu[0][0]']    
                                                                                                  
 conv4_block5_3_bn (BatchNormal  (None, 8, 3, 1024)  4096        ['conv4_block5_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_add (Add)         (None, 8, 3, 1024)   0           ['conv4_block4_out[0][0]',       
                                                                  'conv4_block5_3_bn[0][0]']      
          

 conv5_block2_2_relu (Activatio  (None, 4, 2, 512)   0           ['conv5_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 4, 2, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 4, 2, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 4, 2, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
          

In [ ]:
#Defining training parameters and fitting the model
model.compile(optimizer='adam',loss=weighted_categorical_crossentropy([0.5,0.75,2.5,4]),metrics=[tf.keras.metrics.RootMeanSquaredError()])


history = model.fit(train_generator, epochs = 15)

Epoch 1/15
1575/1575 [==============================] - 867s 547ms/step - loss: 1.1384 - root_mean_squared_error: 0.3555
Epoch 2/15
1575/1575 [==============================] - 828s 526ms/step - loss: 1.0622 - root_mean_squared_error: 0.3462
Epoch 3/15
1575/1575 [==============================] - 825s 524ms/step - loss: 0.9976 - root_mean_squared_error: 0.3375
Epoch 4/15
1575/1575 [==============================] - 1905s 1s/step - loss: 0.9003 - root_mean_squared_error: 0.3231
Epoch 5/15
1575/1575 [==============================] - 836s 531ms/step - loss: 0.8222 - root_mean_squared_error: 0.3122
Epoch 6/15
1575/1575 [==============================] - 828s 526ms/step - loss: 0.7379 - root_mean_squared_error: 0.2998
Epoch 7/15
1575/1575 [==============================] - 1056s 671ms/step - loss: 0.6575 - root_mean_squared_error: 0.2857
Epoch 8/15
1575/1575 [==============================] - 908s 577ms/step - loss: 0.5745 - root_mean_squared_error: 0.2687
Epoch 9/15
1575/1575 [===========

In [15]:
from sklearn.metrics import accuracy_score
#Testing the model
Y_pred = model.predict(test_generator).round()
#Y_pred = np.argmax(Y_pred ,axis =1)


776/776 [==============================] - 93s 117ms/step


In [16]:
print("Accuracy:")
m = tf.keras.utils.to_categorical(test_generator.classes)
acc = accuracy_score(m, Y_pred)
print(acc)
#print(Y_pred)
#print(m)

Accuracy:
0.30749395648670425


In [25]:
prediction = tf.keras.metrics.RootMeanSquaredError()
prediction.update_state(m, Y_pred)
prediction.result().numpy()

0.5582146

In [ ]:
y_true= y_tests
y_predictions= (proba_predictions > 0.5).astype('int32')
print(classification_report(y_true,y_predictions ))

In [26]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])
plt.title('model accuracy and loss')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Accuracy', 'Loss'], loc='upper left')
plt.show()

NameError: name 'plt' is not defined